In [17]:
import iris
import numpy as np

\begin{equation}
\int_{0}^{t} FA \; dt = \bar{F} \int_{0}^{t} A' \; dt \; + \; \bar{A} \int_{0}^{t} F' \; dt 
\end{equation}

In [3]:
evap_area_file = '/g/data/r87/dbi599/CMIP6/DAMIP/CSIRO/ACCESS-ESM1-5/hist-GHG/r1i1p1f1/Ayr/evspsbl/gn/v20200615/evspsbl-pe-region-sum-anomaly_Ayr_ACCESS-ESM1-5_hist-GHG_r1i1p1f1_gn_185001-202012-cumsum.nc'
evap_bar_file = '/g/data/r87/dbi599/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/piControl/r1i1p1f1/Ayr/evspsbl/gn/v20191214/evspsbl-per-m2-pe-region-sum_Ayr_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_010101-100012.nc'
evap_dashed_file = '/g/data/r87/dbi599/CMIP6/DAMIP/CSIRO/ACCESS-ESM1-5/hist-GHG/r1i1p1f1/Ayr/evspsbl/gn/v20200615/evspsbl-per-m2-pe-region-sum-anomaly_Ayr_ACCESS-ESM1-5_hist-GHG_r1i1p1f1_gn_185001-202012-cumsum.nc'
area_bar_file = '/g/data/r87/dbi599/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/piControl/r1i1p1f1/Ayr/areacella/gn/v20191214/areacella-pe-region-sum_Ayr_ACCESS-ESM1-5_piControl_r1i1p1f1_gn_010101-100012.nc'
area_dashed_file = '/g/data/r87/dbi599/CMIP6/DAMIP/CSIRO/ACCESS-ESM1-5/hist-GHG/r1i1p1f1/Ayr/areacella/gn/v20200615/areacella-pe-region-sum-anomaly_Ayr_ACCESS-ESM1-5_hist-GHG_r1i1p1f1_gn_185001-202012-cumsum.nc'

In [9]:
evap_area_cube = iris.load_cube(evap_area_file, 'water_evapotranspiration_flux')
evap_bar_cube = iris.load_cube(evap_bar_file, 'water_evapotranspiration_flux')
evap_dashed_cube = iris.load_cube(evap_dashed_file, 'water_evapotranspiration_flux')
area_bar_cube = iris.load_cube(area_bar_file, 'cell_area')
area_dashed_cube = iris.load_cube(area_dashed_file, 'cell_area')

In [18]:
def get_time_integral(cube, variable):
    """Get time integral"""

    start = cube[0, 0:5, -1].data
    value = cube[-1, 0:5, -1].data - start
    
    return value


def get_average(cube, variable, ntime):
    """Get average"""
    
    data = cube.data[0:ntime, 0:5, -1]
    average = np.mean(data, axis=0)
    
    return average

In [21]:
evap_area_integral = get_time_integral(evap_area_cube, 'water_evapotranspiration_flux')
evap_dashed_integral = get_time_integral(evap_dashed_cube, 'water_evapotranspiration_flux')
area_dashed_integral = get_time_integral(area_dashed_cube, 'cell_area')

In [24]:
evap_bar = get_average(evap_bar_cube, 'water_evapotranspiration_flux', evap_area_cube.shape[0])
area_bar = get_average(area_bar_cube, 'cell_area', evap_area_cube.shape[0])

In [36]:
evap_area_integral.data

array([-8.97865292e+16,  4.66662497e+17,  1.17329324e+17,  2.89853212e+17,
        6.27760929e+16])

In [28]:
(evap_bar * area_dashed_integral) + (area_bar * evap_dashed_integral) 

masked_array(data=[-7.39432861e+20,  3.04358774e+21,  1.29801805e+20,
                    1.44941548e+21,  4.96638322e+20],
             mask=False,
       fill_value=1e+20)

In [29]:
evap_bar

masked_array(data=[3370543.40286642, 6897239.89663527, 4528934.43981984,
                   5826988.20925174, 3698375.96108294],
             mask=False,
       fill_value=1e+20)

In [31]:
area_dashed_integral

masked_array(data=[-9.83168332e+13,  1.51489168e+14, -4.12164668e+13,
                    4.58853861e+13, -5.81600206e+13],
             mask=False,
       fill_value=1e+20)

In [32]:
area_bar

masked_array(data=[1.02978399e+14, 1.11381543e+14, 8.05425591e+13,
                   1.09618489e+14, 1.05393551e+14],
             mask=False,
       fill_value=1e+20)

In [34]:
evap_dashed_integral

masked_array(data=[-3962498.0798631 , 17944899.63129173,
                    3929208.16327426, 10783234.4899975 ,
                    6753126.16717428],
             mask=False,
       fill_value=1e+20)